In [ ]:
!pip install -q timm
!pip install -qq efficientnet_pytorch

### Special thanks to Grandmaster Abhishek thakur, for super useful pytorch trainer

In [ ]:
tez_path = '../input/tez-modified-tqdm/'

In [ ]:
import sys
sys.path.append(tez_path)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as immg
import matplotlib.ticker as ticker
import seaborn as sns
from PIL import Image
import matplotlib.image as immg
import os,gc,random
import albumentations
import tez
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
from efficientnet_pytorch import EfficientNet

In [ ]:
os.mkdir('/kaggle/working/train/')

In [ ]:
!unzip -qq '../input/band-pass-traintest-g2net-audio-wave-data/TrainG2NET.zip' -d '/kaggle/working/train/'

In [ ]:
os.mkdir('/kaggle/working/test/')

In [ ]:
!unzip -qq '../input/band-pass-traintest-g2net-audio-wave-data/TestG2NET.zip' -d '/kaggle/working/test/'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Loading data

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
test = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

def get_test_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/test/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

In [ ]:
train['file_path'] = train['id'].apply(lambda x:"/kaggle/working/train/"+str(x)+".png")

## Stratified Kfold

In [ ]:
from sklearn.model_selection import train_test_split,StratifiedKFold

In [ ]:
train_df = train.copy()
train_df['kfold'] = -1
y = train_df['target'].values
kf = StratifiedKFold(n_splits=200,random_state = 2021,shuffle = True)
for fold ,(trn_,val_ )in enumerate(kf.split(X=train_df,y=y)):
    train_df.loc[val_,'kfold'] = fold

In [ ]:
# train_df.to_csv('cnn1d_cqt_train_kfold.csv',index=False)

In [ ]:
# train_df = pd.read_csv('../input/effnetb0-3ch3-g2net/cnn1d_cqt_train_kfold.csv')

In [ ]:
train_df[train_df.kfold==1].shape,train_df[train_df.kfold!=1].shape

In [ ]:
train_df.head()

In [ ]:
FOLD = 1

In [ ]:
df_train = train_df[train_df['kfold']!=FOLD].reset_index(drop=True)
df_valid = train_df[train_df['kfold']==FOLD].reset_index(drop=True)
train_targets = df_train['target'].values
valid_targets = df_valid['target'].values

In [ ]:
df_train.shape[0],df_valid.shape[0]

## Dataset

In [ ]:
class G2NetDataset:
    def __init__(self,paths,targets):
        
        self.paths = paths
        self.targets = targets
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,item):
        
        targs = self.targets[item]
        image = Image.open(self.paths[item])
        image = np.array(image)/255.0
        image123 = np.array([image[:,:129],image[:,129:129*2],image[:,129*2:]])
        image_tensor = torch.tensor(image123,dtype=torch.float)
        
        return {"image": image_tensor,
                "targets": torch.tensor(targs,dtype=torch.float)}

In [ ]:
train_dataset = G2NetDataset(df_train.file_path,train_targets)
valid_dataset = G2NetDataset(df_valid.file_path,valid_targets)

In [ ]:
img  = train_dataset[0]['image']
tar = train_dataset[0]['targets']

In [ ]:
import sklearn.metrics as sklm
import torch.optim as optim

## Model G2Net ResNest50d

In [ ]:
class G2NetModel(tez.Model):
    def __init__(self):
        super().__init__()
        
        self.net = timm.create_model('resnest50d',pretrained=True)
        #self.net = EfficientNet.from_pretrained("efficientnet-b4")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1000,1)
        
        self.step_scheduler_after = "epoch"
        self.step_scheduler_metric = "valid_rocauc"
        

    def monitor_metrics(self, outputs, targets):
        outputs = torch.sigmoid(outputs).cpu().detach().numpy() 
        targets = targets.cpu().detach().numpy()
        try:
            res = sklm.roc_auc_score(targets, outputs)
        except:
            res = 0.5
        return {"rocauc": res}

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt

    def fetch_scheduler(self):
        rlr = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            verbose=True,
            factor=0.7,
            mode="max",
            patience=2,
            threshold=0.01,)
        return rlr

    def forward(self, image, targets=None):
        
        #batch_size, _, _, _ = image.shape
        x = self.net(image)
        #x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        
        if targets is not None:
            loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, {}

In [ ]:
model = G2NetModel()

In [ ]:
from tez.callbacks import EarlyStopping
es = EarlyStopping(
    monitor="valid_rocauc", model_path="model.bin", patience=5, mode="max"
)

In [ ]:
print('Starting Model Training:')

## Start Training

In [ ]:
model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs = 128,
        valid_bs = 128,
        device = "cuda",
        epochs = 8,
        callbacks = [es],
        fp16 = True)

## Test Dataset & Load Best Model

In [ ]:
class G2NetModel(tez.Model):
    def __init__(self):
        super().__init__()
        self.net = timm.create_model('resnest50d',pretrained=False)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1000,1)
    def forward(self, image):
        x = self.net(image)
        outputs = self.out(self.dropout(x))
        return outputs, None, {}

In [ ]:
model = G2NetModel()

In [ ]:
model.load(os.path.join('/kaggle/working', "model.bin"))

In [ ]:
sub = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [ ]:
sub.head()

In [ ]:
ts_filepath = sub['id'].apply(lambda x:'/kaggle/working/test/'+str(x)+'.png').values

In [ ]:
class TestDataset:
    def __init__(self,paths):
        
        self.paths = paths
        self.targets = None
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,item):
        
        image = Image.open(self.paths[item])
        image = np.array(image)/255.0
        image123 = np.array([image[:,:129],image[:,129:129*2],image[:,129*2:]])
        image_tensor = torch.tensor(image123,dtype=torch.float)
        
        return {"image": image_tensor}

In [ ]:
test_dataset = TestDataset(ts_filepath)

## Prediction on Test Set

In [ ]:
preds = model.predict(
    test_dataset, batch_size=64, n_jobs=2, device="cuda"
)
temp_preds = None
for p in preds:
    if temp_preds is None:
        temp_preds = p
    else:
        temp_preds = np.vstack((temp_preds, p))

In [ ]:
sub['target'] = torch.from_numpy(temp_preds).sigmoid().flatten().numpy()

In [ ]:
sub.to_csv('submission_3ch.csv',index=False)

In [ ]:
!rm -r '/kaggle/working/test/'

In [ ]:
!rm -r '/kaggle/working/train/'